
```
pip install git+https://github.com/josejg/instruction_following_eval.git
```

In [5]:
from instruction_following_eval import default_examples, instruction_following_eval
import pandas as pd
import jsonlines
import os
from pprint import pprint
from glob import glob

In [8]:
files = glob("../outputs/*/*/ifeval.json")
dfs = []

for file in files:
    examples = list(jsonlines.open(file))
    generator = examples[0]['generator']
    for example in examples:
        # example["response"] = example.pop("output")
        example.pop("generator")

    accuracy = instruction_following_eval(examples)[0]['average']
    pprint(accuracy)
    dfs.append({
        "generator": generator,
        "accuracy": accuracy
    })
    
df = pd.DataFrame(dfs)
df

0.27912535420943224


,generator,accuracy
0,heegyu/TinyLlama__TinyLlama-1.1B-intermediate-...,0.279125
